In [8]:
from tvDatafeed import TvDatafeed, Interval
tv = TvDatafeed()
import numpy as np
import pandas as pd
import talib
import time

In [3]:
df = pd.read_csv("ind_nifty100list.csv")
stocks = df['Symbol'].unique().tolist() #Sending Output string
stocks[12] = 'BAJAJ_AUTO'
stocks[62] = 'M_M'

In [4]:
def label_change(row):
 if row['sign_change']:
  if row['sign'] == 1:
   return 'Buy' #Negative to Positive
  else:
   return 'Sell' #Positive to Negative
 return None

In [5]:
all_cases_df = pd.DataFrame()

In [ ]:
def process_stock(symbol):
    try:
        stock_history = tv.get_hist(symbol=symbol, exchange='NSE', interval=Interval.in_daily, n_bars=300)
        close_values = stock_history['close']
        low_values = stock_history['low']
        macd_stock = talib.MACD(stock_history['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        ema_stock = talib.EMA(stock_history['close'], timeperiod=9)
        df = pd.DataFrame({'time': close_values.index, 'close_price': close_values.values, 'macd-signal': macd_stock[2].values, 'ema9': ema_stock.values})
        # ema_max50
        df['ema9_max'] = df['ema9'].rolling(window=50, min_periods=1).max()
        df['ema9_min'] = df['ema9'].rolling(window=50, min_periods=1).min()
        ema_max_50 = df[df['ema9'] == df['ema9_max']]  # ema-max-50days
        ema_min_50 = df[df['ema9'] == df['ema9_min']]  # ema-min-50days
        df['is_50_day_high'] = df['ema9'] == df['ema9_max']  # Boolean column
        df['is_50_day_low'] = df['ema9'] == df['ema9_min']  # Boolean column
        df['macd-signal'] = df['macd-signal'].fillna(method='ffill').fillna(method='bfill')
        df['sign'] = df['macd-signal'].apply(lambda x: 1 if x > 0 else -1)
        df['sign_change'] = df['sign'] != df['sign'].shift(1)
        df['action'] = df.apply(label_change, axis=1)
        df['stock_name'] = symbol
        buy_cases_df = df[(df['is_50_day_high'] == True) & (df['action'] == 'Buy')][['stock_name', 'time', 'close_price', 'action', 'ema9_max', 'macd-signal']]# Collecting all buy cases
        sell_cases_df = df[(df['is_50_day_low'] == True) & (df['action'] == 'Sell')][['stock_name', 'time', 'close_price', 'action', 'ema9_min', 'macd-signal']]# Collecting all sell cases
        combined_df = pd.concat([buy_cases_df, sell_cases_df])# Combine buy_cases_df and sell_cases_df
        combined_df = combined_df.sort_values(by='time', ascending=True)# Sort combined dataframe by time in descending order
        combined_df.reset_index(drop=True, inplace=True)# Reset index for clean output
        return combined_df
    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        return None
all_cases_df = pd.DataFrame()
retries = 5

for i in range(len(stocks)):
    symbol = stocks[i]
    for attempt in range(retries):
        result_df = process_stock(symbol)
        if result_df is not None:
            all_cases_df = pd.concat([all_cases_df, result_df], ignore_index=True)
            break
        else:
            print(f"Retrying {symbol}... Attempt {attempt + 1} of {retries}")
            time.sleep(1)  

In [ ]:
all_cases_df

In [8]:
all_cases_df.to_csv('ema-macd-actions.csv', index=False)